# Using soup

In [1]:
import scraping_class
logfile = 'log.txt' ## name your log file.
connector = scraping_class.Connector(logfile)

In [2]:
from bs4 import BeautifulSoup
import requests
main_url = 'https://kurser.ku.dk/archive'
response = connector.get(main_url,'year_archives_links')
raw_html = response[0].text
# Collecting the soup
soup = BeautifulSoup(raw_html,'html.parser')

In [3]:
from tqdm import tqdm_notebook

### UCPH  Courses - Step 1: Collecting links for the yearly archives

In [4]:
# Collecting all the hyperlinks from the mainpage
all_links = [a.get('href') for a in soup.find_all('a', href=True)]

# Saving all the hyperlinks which contains the 'archive'
endpoint_links = [s for s in all_links if "archive" in s]

# Removing 'archive' from the endpoint links since the word is already in the main_url
endpoint_links = [w[7:] for w in endpoint_links]
endpoint_links

['/2017-2018', '/2016-2017', '/2015-2016', '/2014-2015', '/2013-2014']

In [5]:
# Creating the right links to scrape from the yearly archives
desired_links_year = []
for i in range(0,len(endpoint_links)):
    url = main_url + endpoint_links[i]
    desired_links_year.append(url)
desired_links_year

['https://kurser.ku.dk/archive/2017-2018',
 'https://kurser.ku.dk/archive/2016-2017',
 'https://kurser.ku.dk/archive/2015-2016',
 'https://kurser.ku.dk/archive/2014-2015',
 'https://kurser.ku.dk/archive/2013-2014']

### UCPH Courses - Step 2: Collceting links for each Study Board

In [6]:
soup_2 = []
all_links_2 = []
endpoint_links_2 = []
endpoint_links_2_final = []
desired_links_study_board = []

for i in range(0,len(desired_links_year)):
    
    main_url_2 = desired_links_year[i]
    print(main_url_2)
    
    response = connector.get(main_url_2,'year_archives_links')
    raw_html_2 = response[0].text
    # Collecting the soup
    soup_2.append(BeautifulSoup(raw_html_2,'html.parser'))
    # soup_2 is a list of lists, each lists contains html code for each yearly archive.
    
    # Collecting all the hyperlinks from the mainpage
    all_links_2.append([a.get('href') for a in soup_2[i].find_all('a', href=True)])
    
    # Saving the given year, this step only needs to be run once. 
    if i == 0:
        given_year = [w[29:] for w in desired_links_year]
   
    # Saving all the hyperlinks which contains the given year, these are the links for the Study boards
    endpoint_links_2.append([s for s in all_links_2[i] if given_year[i] in s])
    
    # Removing the year from the endpoint_links_2 links since the word is already in the desired_links_year
    endpoint_links_2_final.append([b[9:] for b in endpoint_links_2[i]])
    
    # Creating the right links to scrape from the study board archives
    for k in range(0,len(endpoint_links_2_final[i])): 
        
        # Getting elements in the lists out as strings in the correct way
        str_studyboard = ''.join(endpoint_links_2_final[i][0+k:1+k])
        
        desired_links_study_board.append(desired_links_year[i] + str_studyboard)
        
soup_2;
all_links_2;
given_year;
endpoint_links_2;
endpoint_links_2_final;
desired_links_study_board;

https://kurser.ku.dk/archive/2017-2018
https://kurser.ku.dk/archive/2016-2017
https://kurser.ku.dk/archive/2015-2016
https://kurser.ku.dk/archive/2014-2015
https://kurser.ku.dk/archive/2013-2014


In [7]:
len(desired_links_study_board)

192

In [8]:
desired_links_study_board[:5]

['https://kurser.ku.dk/archive/2017-2018/888',
 'https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0010',
 'https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0021',
 'https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0044',
 'https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0009']

### UCPH Courses - Step 3: Collecting links for each course

In [9]:
desired_links_study_board_rev = []

# We have to edit til disered_links_study_board because the course url do not contain the study board url.
for q in range(0,len(desired_links_study_board)):
    desired_links_study_board_rev.append(desired_links_study_board[q][0:38])

In [10]:
soup_3 = []
all_links_3 = []
endpoint_links_3 = []
endpoint_links_3_final=[]
desired_links_courses = []

#Looping through all studyboards in a given year and collecting the course links
for i in range(0,len(desired_links_study_board)):
    main_url_3 = desired_links_study_board[i]
    print(main_url_3)
    
    response = connector.get(main_url_3,'study_board_years_archives_links')
    raw_html_3 = response[0].text
    # Collecting the soup
    soup_3.append(BeautifulSoup(raw_html_3,'html.parser'))
    print(len(soup_3)) 
        
    # Collecting all the hyperlinks from the course page from a given study board in a given year
    all_links_3.append([a.get('href') for a in soup_3[i].find_all('a', href=True)])
    
    # Saving all the hyperlinks which contains the given year, these are the links for the Study boards
    endpoint_links_3.append([s for s in all_links_3[i] if '/course/' in s])
    
    # Removing the year and studyboard from endpoint_links_3 
    endpoint_links_3_final.append([b[18:] for b in endpoint_links_3[i]]) 
    
    # Running through each study board and collecting the course urls.
    for k in range(0,len(endpoint_links_3_final[i])):
        
        # Getting elements in the lists out as strings in the correct way
        # Desired_links_courses is a list containing all course links. 
        desired_links_courses.append(desired_links_study_board_rev[i] + ''.join(endpoint_links_3_final[i][k]))


https://kurser.ku.dk/archive/2017-2018/888
1
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0010
2
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0021
3
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0044
4
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0009
5
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_PB_01
6
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_SU_0031
7
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0012
8
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0033
9
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0025
10
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0019
11
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD__BI_0031
12
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0028
13
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0027
14
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0029
15
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0034
16
https://kurser.ku.dk/archive/2017-2018/STUDYBOARD_0030
17
https://kurser.ku.dk/archiv

https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_MA_0013
142
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0030
143
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0031
144
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_PHD_0010
145
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0001
146
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0005
147
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_MA_0014
148
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0006
149
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0016
150
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0091
151
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0003
152
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0203
153
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0014
154
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0004
155
https://kurser.ku.dk/archive/2014-2015/STUDYBOARD_0023
156
https://kurser.ku.dk/archive/2013-2014/STUDYBOARD_0010
157
https://kurser.ku.dk/archive/2013-2014/STUDYBO

In [12]:
print(len(desired_links_courses))
print(len(set(desired_links_courses)))
#we have no dublicates - which makes sence.

20035
20035


In [13]:
desired_links_courses[0]

'https://kurser.ku.dk/archive/2017-2018/course/UCPHAUK18U'

### UCPH Courses - Collecting the course ID's

In [14]:
course_ids = []

for i in range(0,len(desired_links_courses)):
    course_ids.append(desired_links_courses[i][46:])


print(len(course_ids))

course_ids[0]

20035


'UCPHAUK18U'

In [15]:
# A set object is an unordered collection of distinct hashable objects - we convert the set back to a list.
unique_course_ids = list(set(course_ids))
len(unique_course_ids)
# We see that we have some dublicates in the course_ids list. This is becuase some courses do not change ID from year to year.

11126

In [16]:
#The first course in the KU archieve is at CBS not KU and is of wierd formatting. We drop the course.
print(course_ids[0])
print(unique_course_ids[0])
print(desired_links_courses[0])

course_ids.remove('UCPHAUK18U')
unique_course_ids.remove('UCPHAUK18U')
desired_links_courses.remove('https://kurser.ku.dk/archive/2017-2018/course/UCPHAUK18U')

UCPHAUK18U
HARB01282U
https://kurser.ku.dk/archive/2017-2018/course/UCPHAUK18U


### UCPH Courses - Scraping each course 

In [17]:
import pandas as pd
import numpy as np
import html5lib

In [28]:
def create_df(url):
    # Requesting URL with get
    response = connector.get(url,'course_links')
    
    # Getting course_id and year direct from URL
    course_id = url.split('/')[-1]
    year = url.split('/')[-3]

    # Converting to soup
    raw_html = response[0].text
    soup = BeautifulSoup(raw_html,'lxml')

    # Info used to scrabe certain tables (exam and on in the right corner)
    info = soup.find_all("dl", {'class':'dl-horizontal'})
    
    # Generating error list if course page is out of match
    error = []

    # Only selecting courses with 2 tabels i.e. exactly 1 exam and the info bar to the right
    # Then appending information of each side by 'dt' and 'dd' and pairing them
    if len(info) == 2:
        comp_info = pd.DataFrame()
        cleaned_id_text = ['Course Name']
        for i in info[0].find_all('dt'):
            cleaned_id_text.append(i.text)
        cleaned_id__attrb_text = [soup.find('h1',{'class':"courseTitle"}).text]
        for i in info[0].find_all('dd'):
            cleaned_id__attrb_text.append(i.text)
        
        # If there is a blank space the lists can't be paired. Therfore if lengths dosen't match, list will be created by siblings if sibling isen't on the initial side.
        if len(info[1].find_all('dt')) == len(info[1].find_all('dd')):
            for i in info[1].find_all('dt'):
                cleaned_id_text.append(i.text)
            for i in info[1].find_all('dd'):
                cleaned_id__attrb_text.append(i.text)
        else:
            for i in info[1].find_all('dt'):
                cleaned_id_text.append(i.text)
            for i in info[1].find_all('dt'):
                if i.next_sibling in info[1].find_all('dt'):
                    cleaned_id__attrb_text.append('')
                else:
                    cleaned_id__attrb_text.append(i.next_sibling.text)

        # Appending year from URL
        cleaned_id_text.append('Year')
        cleaned_id__attrb_text.append(year)
        
        # Appending course id form URL
        cleaned_id_text.append('Course id')
        cleaned_id__attrb_text.append(course_id)
        
        # Scrabing data from workload table
        work = soup.find('ul',{'class':'list-unstyled workload clearfix'})
        if work != None:
            workfi = work.find_all('li')
            # Geting 1 list of element, therfore has to be paire 1-3 2-4 etc.
            for i in range(len(workfi)):
                if i % 2 ==0:
                    cleaned_id_text.append(workfi[i].text)
                else: 
                    cleaned_id__attrb_text.append(workfi[i].text)

        # Appending content
        if soup.find_all('div',{'class':"course-content"}) != []:
            cleaned_id_text.append(soup.find_all('div',{'class':"course-content"})[0].find('a').text)
            cleaned_id__attrb_text.append(soup.find_all('div',{'class':"course-content"})[0].find('div').text)
        
        # Appending learning outcome
        if soup.find_all('div',{'class':"course-description"}) != []:
            cleaned_id_text.append(soup.find_all('div',{'class':"course-description"})[0].find('a').text)
            cleaned_id__attrb_text.append(soup.find_all('div',{'class':"course-description"})[0].find('div').text)

        comp_info['Id'] = cleaned_id_text
        comp_info['Attribute'] = cleaned_id__attrb_text
    else:
        error.append(url)
        comp_info = []
        
    return(comp_info, error)

In [29]:
dct_raw = dict()
wierd_urls = []

for i in tqdm_notebook(range(0,len(desired_links_courses))):
    url = desired_links_courses[i]
    print(desired_links_courses[i])
    dftest,error=create_df(url)
    
    # No need for a column called Id. We just use the Id column as the index. For now.
    # Also, we rename the other column ('Attribute') to the course url since we will use this as the key in the dict. 
    if type(dftest) != list:
        dftest=dftest.set_index('Id').rename(columns = {'Attribute': desired_links_courses[i]})
        
        # Converting the dataframe to a dictionary
        dct_temp = dict()
        dct_temp = dftest.to_dict()
    
        # Adding to dict
        dct_raw.update(dct_temp)
        
    else: 
        wierd_urls.append(error)
    

https://kurser.ku.dk/archive/2017-2018/course/AANA17100U
https://kurser.ku.dk/archive/2017-2018/course/AANA17101U
https://kurser.ku.dk/archive/2017-2018/course/AANA17102U
https://kurser.ku.dk/archive/2017-2018/course/AANA17103U
https://kurser.ku.dk/archive/2017-2018/course/AANA17104U
https://kurser.ku.dk/archive/2017-2018/course/AANA17105U
https://kurser.ku.dk/archive/2017-2018/course/AANA20003U
https://kurser.ku.dk/archive/2017-2018/course/AANA20004U
https://kurser.ku.dk/archive/2017-2018/course/AANB05023U
https://kurser.ku.dk/archive/2017-2018/course/AANB05024U
https://kurser.ku.dk/archive/2017-2018/course/AANB05044U
https://kurser.ku.dk/archive/2017-2018/course/AANB05059U
https://kurser.ku.dk/archive/2017-2018/course/AANB05081U
https://kurser.ku.dk/archive/2017-2018/course/AANB05090U
https://kurser.ku.dk/archive/2017-2018/course/AANB05091U
https://kurser.ku.dk/archive/2017-2018/course/AANB05093U
https://kurser.ku.dk/archive/2017-2018/course/AANB11002U
https://kurser.ku.dk/archive/20

https://kurser.ku.dk/archive/2017-2018/course/JJUA55196U
https://kurser.ku.dk/archive/2017-2018/course/JJUA55197U
https://kurser.ku.dk/archive/2017-2018/course/JJUA55198U
https://kurser.ku.dk/archive/2017-2018/course/JJUA55199U
https://kurser.ku.dk/archive/2017-2018/course/JJUA55200U
https://kurser.ku.dk/archive/2017-2018/course/JJUA55202U
https://kurser.ku.dk/archive/2017-2018/course/JJUA55203U
https://kurser.ku.dk/archive/2017-2018/course/JJUA55204U
https://kurser.ku.dk/archive/2017-2018/course/JJUA55205U
https://kurser.ku.dk/archive/2017-2018/course/JJUA55206U
https://kurser.ku.dk/archive/2017-2018/course/JJUA55207U
https://kurser.ku.dk/archive/2017-2018/course/JJUA55208U
https://kurser.ku.dk/archive/2017-2018/course/JJUA55209U
https://kurser.ku.dk/archive/2017-2018/course/JJUA55210U
https://kurser.ku.dk/archive/2017-2018/course/JJUA55212U
https://kurser.ku.dk/archive/2017-2018/course/JJUA55213U
https://kurser.ku.dk/archive/2017-2018/course/JJUA55214U
https://kurser.ku.dk/archive/20

https://kurser.ku.dk/archive/2017-2018/course/SODA15062U
https://kurser.ku.dk/archive/2017-2018/course/SODA15063E
https://kurser.ku.dk/archive/2017-2018/course/SODA15064U
https://kurser.ku.dk/archive/2017-2018/course/SODA15065U
https://kurser.ku.dk/archive/2017-2018/course/SODA15066U
https://kurser.ku.dk/archive/2017-2018/course/SODA15067U
https://kurser.ku.dk/archive/2017-2018/course/SODA15068U
https://kurser.ku.dk/archive/2017-2018/course/SODA15069U
https://kurser.ku.dk/archive/2017-2018/course/SODA15072U
https://kurser.ku.dk/archive/2017-2018/course/SODA15073U
https://kurser.ku.dk/archive/2017-2018/course/SODA15077U
https://kurser.ku.dk/archive/2017-2018/course/SODA15078U
https://kurser.ku.dk/archive/2017-2018/course/SODA15079U
https://kurser.ku.dk/archive/2017-2018/course/SODA15080U
https://kurser.ku.dk/archive/2017-2018/course/SODB16011E
https://kurser.ku.dk/archive/2017-2018/course/SODB16011U
https://kurser.ku.dk/archive/2017-2018/course/SODB16012U
https://kurser.ku.dk/archive/20

https://kurser.ku.dk/archive/2017-2018/course/AØKA08178U
https://kurser.ku.dk/archive/2017-2018/course/AØKA08184U
https://kurser.ku.dk/archive/2017-2018/course/AØKA08186U
https://kurser.ku.dk/archive/2017-2018/course/AØKA08189U
https://kurser.ku.dk/archive/2017-2018/course/AØKA08191U
https://kurser.ku.dk/archive/2017-2018/course/AØKA08197U
https://kurser.ku.dk/archive/2017-2018/course/AØKA08202U
https://kurser.ku.dk/archive/2017-2018/course/AØKA08208U
https://kurser.ku.dk/archive/2017-2018/course/AØKA08209U
https://kurser.ku.dk/archive/2017-2018/course/AØKA08212U
https://kurser.ku.dk/archive/2017-2018/course/AØKA08214U
https://kurser.ku.dk/archive/2017-2018/course/AØKA08216U
https://kurser.ku.dk/archive/2017-2018/course/AØKA08218U
https://kurser.ku.dk/archive/2017-2018/course/AØKA08219U
https://kurser.ku.dk/archive/2017-2018/course/AØKA08221U
https://kurser.ku.dk/archive/2017-2018/course/AØKA08223U
https://kurser.ku.dk/archive/2017-2018/course/AØKA08225U
https://kurser.ku.dk/archive/20

https://kurser.ku.dk/archive/2017-2018/course/NPIPNBIMSC
https://kurser.ku.dk/archive/2017-2018/course/NPIPNEXMSC
https://kurser.ku.dk/archive/2017-2018/course/NPIPPLEMSC
https://kurser.ku.dk/archive/2017-2018/course/NPIPSNMMSC
https://kurser.ku.dk/archive/2017-2018/course/NVIRKBIOBA
https://kurser.ku.dk/archive/2017-2018/course/NVIRKCHEBA
https://kurser.ku.dk/archive/2017-2018/course/NVIRKDIKBA
https://kurser.ku.dk/archive/2017-2018/course/NVIRKFOOBA
https://kurser.ku.dk/archive/2017-2018/course/NVIRKIFRBA
https://kurser.ku.dk/archive/2017-2018/course/NVIRKIGNBA
https://kurser.ku.dk/archive/2017-2018/course/NVIRKINDBA
https://kurser.ku.dk/archive/2017-2018/course/NVIRKMATBA
https://kurser.ku.dk/archive/2017-2018/course/NVIRKNBIBA
https://kurser.ku.dk/archive/2017-2018/course/NVIRKNEXBA
https://kurser.ku.dk/archive/2017-2018/course/NVIRKPLEBA
https://kurser.ku.dk/archive/2017-2018/course/NVIRKSNMBA
https://kurser.ku.dk/archive/2017-2018/course/ASOA15013U
https://kurser.ku.dk/archive/20

...

https://kurser.ku.dk/archive/2016-2017/course/AØKA08076U
https://kurser.ku.dk/archive/2016-2017/course/AØKA08077U
https://kurser.ku.dk/archive/2016-2017/course/AØKA08079U
https://kurser.ku.dk/archive/2016-2017/course/AØKA08080U
https://kurser.ku.dk/archive/2016-2017/course/AØKA08081U
https://kurser.ku.dk/archive/2016-2017/course/AØKA08082U
https://kurser.ku.dk/archive/2016-2017/course/AØKA08084U
https://kurser.ku.dk/archive/2016-2017/course/AØKA08087U
https://kurser.ku.dk/archive/2016-2017/course/AØKA08088U
https://kurser.ku.dk/archive/2016-2017/course/AØKA08089U
https://kurser.ku.dk/archive/2016-2017/course/AØKA08091U
https://kurser.ku.dk/archive/2016-2017/course/AØKA08093U
https://kurser.ku.dk/archive/2016-2017/course/AØKA08098U
https://kurser.ku.dk/archive/2016-2017/course/AØKA08100U
https://kurser.ku.dk/archive/2016-2017/course/AØKA08101U
https://kurser.ku.dk/archive/2016-2017/course/AØKA08104U
https://kurser.ku.dk/archive/2016-2017/course/AØKA08107U
https://kurser.ku.dk/archive/20

https://kurser.ku.dk/archive/2016-2017/course/LSLS10056U
https://kurser.ku.dk/archive/2016-2017/course/LSLS10057U
https://kurser.ku.dk/archive/2016-2017/course/LSLS10059U
https://kurser.ku.dk/archive/2016-2017/course/LSLS10060U
https://kurser.ku.dk/archive/2016-2017/course/LSLS10061U
https://kurser.ku.dk/archive/2016-2017/course/LSLS10062U
https://kurser.ku.dk/archive/2016-2017/course/LSLS10063U
https://kurser.ku.dk/archive/2016-2017/course/LSLS10064U
https://kurser.ku.dk/archive/2016-2017/course/LSLS10066U
https://kurser.ku.dk/archive/2016-2017/course/LSLS10070U
https://kurser.ku.dk/archive/2016-2017/course/LSLS10071U
https://kurser.ku.dk/archive/2016-2017/course/LSLS10083U
https://kurser.ku.dk/archive/2016-2017/course/LSLS10084U
https://kurser.ku.dk/archive/2016-2017/course/LSLS10086U
https://kurser.ku.dk/archive/2016-2017/course/LSLS10087U
https://kurser.ku.dk/archive/2016-2017/course/LSLS10088U
https://kurser.ku.dk/archive/2016-2017/course/LSLS10091U
https://kurser.ku.dk/archive/20

https://kurser.ku.dk/archive/2016-2017/course/ASTK15434U
https://kurser.ku.dk/archive/2016-2017/course/ASTK15435U
https://kurser.ku.dk/archive/2016-2017/course/ASTK15436U
https://kurser.ku.dk/archive/2016-2017/course/ASTK15437U
https://kurser.ku.dk/archive/2016-2017/course/ASTK15438U
https://kurser.ku.dk/archive/2016-2017/course/ASTK15439U
https://kurser.ku.dk/archive/2016-2017/course/ASTK15440U
https://kurser.ku.dk/archive/2016-2017/course/ASTK15441U
https://kurser.ku.dk/archive/2016-2017/course/ASTK15442U
https://kurser.ku.dk/archive/2016-2017/course/ASTK15449U
https://kurser.ku.dk/archive/2016-2017/course/ASTK15450U
https://kurser.ku.dk/archive/2016-2017/course/ASTK15451U
https://kurser.ku.dk/archive/2016-2017/course/ASTK15452U
https://kurser.ku.dk/archive/2016-2017/course/ASTK15453U
https://kurser.ku.dk/archive/2016-2017/course/ASTK15454U
https://kurser.ku.dk/archive/2016-2017/course/ASTK15455U
https://kurser.ku.dk/archive/2016-2017/course/ASTK15456U
https://kurser.ku.dk/archive/20

https://kurser.ku.dk/archive/2016-2017/course/NBIK15011U
https://kurser.ku.dk/archive/2016-2017/course/NBIK15013U
https://kurser.ku.dk/archive/2016-2017/course/NBIK15014U
https://kurser.ku.dk/archive/2016-2017/course/NBIK15015U
https://kurser.ku.dk/archive/2016-2017/course/NBIK15016U
https://kurser.ku.dk/archive/2016-2017/course/NBIK15017U
https://kurser.ku.dk/archive/2016-2017/course/NBIK15019U
https://kurser.ku.dk/archive/2016-2017/course/NBIK15020U
https://kurser.ku.dk/archive/2016-2017/course/NBIK16000U
https://kurser.ku.dk/archive/2016-2017/course/NBIK16003U
https://kurser.ku.dk/archive/2016-2017/course/NIFB13002U
https://kurser.ku.dk/archive/2016-2017/course/NIFB13006U
https://kurser.ku.dk/archive/2016-2017/course/NKEA04065U
https://kurser.ku.dk/archive/2016-2017/course/NKEB15001U
https://kurser.ku.dk/archive/2016-2017/course/NMAA04011U
https://kurser.ku.dk/archive/2016-2017/course/NMAB14031U
https://kurser.ku.dk/archive/2016-2017/course/NMAK14003U
https://kurser.ku.dk/archive/20

KeyboardInterrupt: 

In [21]:
print(len(wierd_urls))
#print(wierd_urls[20][0])
print(len(dct_raw))

916
19118


In [24]:
wierd_urls

[['https://kurser.ku.dk/archive/2017-2018/course/AANB11073U'],
 ['https://kurser.ku.dk/archive/2017-2018/course/AANB11074U'],
 ['https://kurser.ku.dk/archive/2017-2018/course/JJUA14124U'],
 ['https://kurser.ku.dk/archive/2017-2018/course/JJUA54029U'],
 ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15016U'],
 ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15019U'],
 ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15022U'],
 ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15025U'],
 ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15031U'],
 ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15043U'],
 ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15048U'],
 ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15049U'],
 ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15050U'],
 ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15051U'],
 ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15053U'],
 ['https://kurser.ku.dk/archive/2017-2018/course/ASOK15

In [20]:
dct = dct_raw

In [21]:
# This code translates keys in danish to the english names. 

# This is a list of urls (keys) which is NOT in the wierd_url list.
lst_dct_keys = list(dct.keys())

for i in range(0,len(dct)):

    # Running through all the keys for a given course i and translating keys
    if 'Hjælpemidler' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Aid'] = dct[lst_dct_keys[i]].pop('Hjælpemidler')
        
    if 'Kategori' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Category'] =dct[lst_dct_keys[i]].pop('Kategori')
        
    if 'Censurform' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Censorship form'] =dct[lst_dct_keys[i]].pop('Censurform')
        
    if 'Kursuskode' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Course code'] =dct[lst_dct_keys[i]].pop('Kursuskode')
        
    if 'Point' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Credit'] =dct[lst_dct_keys[i]].pop('Point')
        
    if 'Varighed' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Duration'] =dct[lst_dct_keys[i]].pop('Varighed')
    
    if 'Eksamen' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Exam'] =dct[lst_dct_keys[i]].pop('Eksamen')
    
    if 'Eksamensperiode' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Exam period'] =dct[lst_dct_keys[i]].pop('Eksamensperiode')
    
    if 'Sprog' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Language'] =dct[lst_dct_keys[i]].pop('Sprog')
    
    if 'Forelæsninger' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Lectures'] =dct[lst_dct_keys[i]].pop('Forelæsninger')    
        
    if 'Niveau' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Level'] =dct[lst_dct_keys[i]].pop('Niveau')
        
    if 'Bedømmelsesform' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Marking scale'] =dct[lst_dct_keys[i]].pop('Bedømmelsesform')
        
    if 'Placering' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Placement'] =dct[lst_dct_keys[i]].pop('Placering')

    if 'Forberedelse' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Preparation'] =dct[lst_dct_keys[i]].pop('Forberedelse')
        
    if 'Reeksamen' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Re-exam'] =dct[lst_dct_keys[i]].pop('Reeksamen')
        
    if 'Skemagruppe' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Schedule'] =dct[lst_dct_keys[i]].pop('Skemagruppe')
        
    if 'Studienævn' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Study board'] =dct[lst_dct_keys[i]].pop('Studienævn')
        
    if 'I alt' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Total'] =dct[lst_dct_keys[i]].pop('I alt')

    if 'Prøveform' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Type of assessment'] =dct[lst_dct_keys[i]].pop('Prøveform')
        
    if 'Krav til indstilling til eksamen' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Exam registration requirements'] =dct[lst_dct_keys[i]].pop('Krav til indstilling til eksamen')
    
    if 'Kursusindhold' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Content'] =dct[lst_dct_keys[i]].pop('Kursusindhold')
    
    if 'Målbeskrivelser' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Learning\r\nOutcome'] =dct[lst_dct_keys[i]].pop('Målbeskrivelser')
        
    if 'Vejledning' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Guidance'] =dct[lst_dct_keys[i]].pop('Vejledning')
        
    if 'Holdundervisning' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Class Exercises'] =dct[lst_dct_keys[i]].pop('Holdundervisning')
        
    if 'Projektarbejde' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Project work'] =dct[lst_dct_keys[i]].pop('Projektarbejde')
        
    if 'Feltarbejde' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Field Work'] =dct[lst_dct_keys[i]].pop('Feltarbejde')
        
    if 'Eksamensforberedelse' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Exam Preparation'] =dct[lst_dct_keys[i]].pop('Eksamensforberedelse')
        
    if 'Studiegrupper' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Study Groups'] =dct[lst_dct_keys[i]].pop('Studiegrupper')
        
    if 'Teoretiske øvelser' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Theory exercises'] =dct[lst_dct_keys[i]].pop('Teoretiske øvelser')
        
    if 'Ekskursioner' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Excursions'] =dct[lst_dct_keys[i]].pop('Ekskursioner')
        
    if 'Klinik' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Clinical practice'] =dct[lst_dct_keys[i]].pop('Klinik')
        
    if 'E-læring' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['E-Learning'] =dct[lst_dct_keys[i]].pop('E-læring')
        
    if 'Kollokvier' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Colloquia'] =dct[lst_dct_keys[i]].pop('Kollokvier')

    if 'Laboratorie' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Laboratory'] =dct[lst_dct_keys[i]].pop('Laboratorie')
        
    if 'Praktiske øvelser' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Practical exercises'] =dct[lst_dct_keys[i]].pop('Praktiske øvelser')
        
    if 'Kursuskapacitet' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Course capacity'] =dct[lst_dct_keys[i]].pop('Kursuskapacitet')
        
    if 'Efter- og videreuddannelse' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Continuing and further education'] =dct[lst_dct_keys[i]].pop('Efter- og videreuddannelse')
        
    if 'Seminarhold' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Class Seminar'] =dct[lst_dct_keys[i]].pop('Seminarhold')
        
    if 'Pris' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Price'] =dct[lst_dct_keys[i]].pop('Pris')
        
    if 'Undervisningsforberedelse' in  dct[lst_dct_keys[i]].keys():
        dct[lst_dct_keys[i]]['Course Preparation'] =dct[lst_dct_keys[i]].pop('Undervisningsforberedelse')

In [22]:
np.save('kurser_scrabe_ALL.npy', dct) 

In [23]:
np.save('wierd_urls_ALL.npy', wierd_urls)

In [24]:
print(wierd_urls)

[['https://kurser.ku.dk/archive/2017-2018/course/AANB11073U'], ['https://kurser.ku.dk/archive/2017-2018/course/AANB11074U'], ['https://kurser.ku.dk/archive/2017-2018/course/JJUA14124U'], ['https://kurser.ku.dk/archive/2017-2018/course/JJUA54029U'], ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15016U'], ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15019U'], ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15022U'], ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15025U'], ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15031U'], ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15043U'], ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15048U'], ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15049U'], ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15050U'], ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15051U'], ['https://kurser.ku.dk/archive/2017-2018/course/ASOA15053U'], ['https://kurser.ku.dk/archive/2017-2018/course/ASOK15802U'], ['https

### UCPH Courses - Changing the dict to DataFrame

In [49]:
import numpy as np
data = np.load('kurser_scrabe_ALL.npy', allow_pickle = True).item()

In [51]:
len(lst_dct_keys)

19119

In [ ]:


data[]